In [ ]:
import numpy as np
import cv2 as cv
import os
import json
from PIL import Image
import matplotlib.pyplot as plt
import time
import torch

from lac.util import color_mask
from lac.perception.segmentation import Segmentation

%load_ext autoreload
%autoreload 2

In [1]:
print(torch.get_default_dtype())

NameError: name 'torch' is not defined

# Hazard detection

Use combination of instance segmentation and depth estimation to identify hazards (rocks)

- Turn left is positive, turn right is negative


## Segmentation

Empirical observation: masks of area >50000 are typically outliers (full ground).


In [13]:
segmentation = Segmentation()

In [ ]:
data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/hazard_2")
# data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/data_collection_1")

i = 1051
FL_gray = cv.imread(os.path.join(data_path, "front_left", f"{i}.png"), cv.IMREAD_GRAYSCALE)
plt.imshow(FL_gray, cmap="gray")
plt.show()

In [ ]:
results, full_mask = segmentation.segment_rocks(Image.fromarray(FL_gray).convert("RGB"))

In [ ]:
FL_rgb = cv.cvtColor(FL_gray, cv.COLOR_GRAY2BGR)
mask_colored = color_mask(full_mask, (1, 0, 0)).astype(FL_rgb.dtype)
overlay = cv.addWeighted(FL_rgb, 1.0, mask_colored, beta=0.5, gamma=0)
plt.imshow(overlay)

In [ ]:
for mask in results["masks"]:
    mask_area = np.sum(mask)
    if mask_area > 1000:
        # Compute pixel centroid
        mask = mask.astype(np.uint8)
        moments = cv.moments(mask)
        cx = int(moments["m10"] / moments["m00"])
        cy = int(moments["m01"] / moments["m00"])
        cv.circle(overlay, (cx, cy), 5, (0, 255, 0), -1)
        # Compute the x bounds of the mask
        x, y, w, h = cv.boundingRect(mask)
        cv.rectangle(overlay, (x, y), (x + w, y + h), (0, 255, 0), 2)
plt.imshow(overlay)

In [40]:
# Find the max area mask with area under 50000
max_area = 0
max_mask = None
for mask in results["masks"]:
    mask_area = np.sum(mask)
    if mask_area > max_area and mask_area < 50000:
        max_area = mask_area
        max_mask = mask

In [ ]:
cx

In [ ]:
offset = 1280 / 2 - cx
print("Offset:", offset)

if offset > 0:
    print("Turn right")
    print("Steering = ", -min(0.6 * ((x + w) - cx) / 100, 0.6))
else:
    print("Turn left")
    print("Steering = ", min(0.6 * (cx - x) / 100, 0.6))
cx - x

## Depth


In [6]:
import os
import cv2 as cv
from PIL import Image
import matplotlib.pyplot as plt

from lac.perception.depth import DepthAnything

In [2]:
depth_model = DepthAnything()

In [ ]:
data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/hazard_2")
# data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/data_collection_1")

i = 1051
FL_gray = cv.imread(os.path.join(data_path, "front_left", f"{i}.png"), cv.IMREAD_GRAYSCALE)
plt.imshow(FL_gray, cmap="gray")
plt.show()

In [4]:
import torch
from transformers import pipeline

checkpoint = "depth-anything/Depth-Anything-V2-base-hf"
pipe = pipeline("depth-estimation", model=checkpoint, device="cuda")

# # Force model outputs to use float32
# pipe.model.to(torch.float32)
pass

In [ ]:
image = Image.open(os.path.join(data_path, "front_left", f"{i}.png"))

In [8]:
torch.set_default_dtype(torch.float32)

In [ ]:
print(torch.get_default_dtype())

In [ ]:
pipe(image)

In [18]:
import os
from PIL import Image
from transformers import pipeline
import torch
import matplotlib.pyplot as plt

In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"
checkpoint = "depth-anything/Depth-Anything-V2-base-hf"
pipe = pipeline("depth-estimation", model=checkpoint, device=device)

In [ ]:
i = 100

image_path = os.path.expanduser(
    "~/LunarAutonomyChallenge/output/data_collection_1/front_left/{}.png".format(i)
)
image_depth_in = Image.open(image_path)

predictions = pipe(image_depth_in)

# Plot image and predicted depth side by side
fig, axes = plt.subplots(1, 2, figsize=(20, 10), gridspec_kw={"wspace": 0, "hspace": 0})
axes[0].imshow(image_depth_in, cmap="gray")
axes[1].imshow(predictions["depth"], cmap="gray")
for ax in axes:
    ax.axis("off")
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

In [ ]:
image = Image.open(os.path.join(data_path, "front_left", f"{i}.png"))
predictions = pipe(image)

# Plot image and predicted depth side by side
fig, axes = plt.subplots(1, 2, figsize=(20, 10), gridspec_kw={"wspace": 0, "hspace": 0})
axes[0].imshow(image, cmap="gray")
axes[1].imshow(predictions["depth"], cmap="gray")
for ax in axes:
    ax.axis("off")
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()